In [35]:
def rec(xin, i=4, res=[]):
    if i >= 0:
#         res = rec(xin, i-1, res)
#         print(xin[i:(i+1),:2])
#         res.append(i)
        rec(xin, i-1, res)
        res[i] = i
    return res

import numpy as np
xin = np.random.rand(4,2)
print(xin)
print()
print(rec(xin))
print(rec(xin))

[[0.13060989 0.80395   ]
 [0.10387511 0.87307183]
 [0.34684092 0.331112  ]
 [0.77307663 0.17550358]]



IndexError: list assignment index out of range

In [1]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

In [28]:
from keras.layers import Flatten, Conv2D, Conv2DTranspose, Dense, Input, Reshape, concatenate, Activation
from keras import Sequential
from keras.models import Model

class FourSplit(Model):
    def __init__(self):
        super(FourSplit, self).__init__()
        self.n_filters = [32, 64]
        self.encoder = self._create_encoder()
        self.decoder = self._create_decoder()
        self.compile()

    def encode(self, x, z=[None, None, None, None], i=3):
        if i >= 0:
#             xin = x[i*14:(i+1)*14, :14]
            xin = x[0:14, :14]
            pred = self.encoder(xin.reshape(-1,14,14,1))
            self.encode(x, z, i-1)
            z[i] = pred
        return z
    
    def decode(self, zs):
        z = tf.concat(zs, 1)
        return self.decoder(z)
        
    def _create_encoder(self):
        return Sequential([
            Input(shape=(14,14,1)),
            Conv2D(filters=self.n_filters[0], kernel_size=(3,3), strides=(2,2), activation='relu', padding='valid'),
            Conv2D(filters=self.n_filters[1], kernel_size=(3,3), strides=(2,2), activation='relu', padding='valid'),
            Conv2D(filters=1, kernel_size=(1,1), strides=(1,1), activation='relu', padding='valid'),
            Flatten()
        ], name='encoder')

    def _create_decoder(self):
        return Sequential([
            Input(shape=(16)),
            Dense(7 * 7 * 64),
            Reshape((7, 7, 64)),
            Conv2DTranspose(filters=self.n_filters[1], kernel_size=(3,3), strides=(2,2), activation='relu', padding='same'),
            Conv2DTranspose(filters=self.n_filters[0], kernel_size=(3,3), strides=(2,2), activation='relu', padding='same'),
            Conv2DTranspose(filters=1, kernel_size=(3,3), padding='same'),
            Activation('sigmoid', name='decoder_out')
        ])

In [29]:
import tensorflow as tf

def train_step(model, x, optimizer):
    with tf.GradientTape() as tape:   # operations inside gradient tape are saved.
        loss = compute_loss(model, x)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

def compute_loss(model, x):
    z = model.encode(x, [])
    x_pred = model.decode(z)
    err = tf.reduce_mean(tf.square(tf.subtract(x_pred, x))) # MSE
    return err

In [30]:
import time
from keras.optimizers import Adam
from keras.metrics import Mean
import numpy as np

fs = FourSplit()
optimizer = Adam(1e-4)

x_train = x_train[:1000]
train_losses, test_losses = [], []
for epoch in range(1, 10):
    loss = Mean()
    start_time = time.time()
    for x in x_train:
        train_step(fs, x, optimizer)
        loss(compute_loss(fs, x))
    end_time = time.time()
    
    print("train loss: {} in {} sec".format(loss.result(), round(end_time-start_time, 1)))
    np.random.shuffle(x_train)
    
#     loss = Mean()
#     for x in x_test:
#         loss(compute_loss(fs, x))
#     test_losses.append(loss.result())

4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4


KeyboardInterrupt: 

In [54]:
from keras.layers import Input, Conv2D, Lambda
import keras.backend as K
from keras.models import Model

class Reshaper(Model):
    def __init__(self, input_shape, output_shape):
        model_in  = Input(shape=input_shape)
        model_out = K.concatenate((K.variable([-1], dtype='int32'), output_shape))   # arbitrary sampling dim
        shaper = Lambda(lambda x: K.reshape(x, model_out))(model_in)       # apply input to output along (in/out) sampling dim
        super(Reshaper, self).__init__(inputs=model_in, outputs=shaper)

In [57]:
reducer = Reshaper([4, 14, 14, 1], [14, 14, 1])
expander = Reshaper([14, 14, 1], [4, 14, 14, 1])

input shape:  (None, 4, 14, 14, 1)
output shape:  (4,)
reshaped:  (None, 14, 14, 1)
input shape:  (None, 14, 14, 1)
output shape:  (5,)
reshaped:  (None, 4, 14, 14, 1)


In [70]:
import numpy as np

r =np.random.rand(2,3,3)
print(r)
reducer = Reshaper([3,3], [3])
reducer(r)

[[[0.74398505 0.94331127 0.71981297]
  [0.83363238 0.32694204 0.20242849]
  [0.2736385  0.83659905 0.15564704]]

 [[0.19741827 0.46016388 0.03830899]
  [0.25210604 0.89179165 0.59514601]
  [0.22459861 0.97671679 0.76810626]]]
input shape:  (None, 3, 3)
output shape:  (2,)
reshaped:  (None, 3)


<tf.Tensor: shape=(6, 3), dtype=float32, numpy=
array([[0.74398506, 0.9433113 , 0.719813  ],
       [0.8336324 , 0.32694206, 0.20242849],
       [0.2736385 , 0.83659905, 0.15564704],
       [0.19741827, 0.4601639 , 0.03830899],
       [0.25210604, 0.89179164, 0.595146  ],
       [0.2245986 , 0.9767168 , 0.7681063 ]], dtype=float32)>

In [77]:
def image_splitter(im, n):
    _, height = im.shape  # W*H
    im = im.reshape(height // n, n, -1, n)  # split in n 2d arrays along cols
    im = im.swapaxes(1, 2)     # restore order: zig-zag
    im = im.reshape(-1, n, n) # n*n 2d arrays with dims (n,n)
    return im

In [79]:
r = np.random.rand(4,4)
print(r)
image_splitter(r, 2)

[[0.56620119 0.08906015 0.02074013 0.10316636]
 [0.24706707 0.18581138 0.30929114 0.36826331]
 [0.23826158 0.23864747 0.22075985 0.93674687]
 [0.32624601 0.52732461 0.492622   0.31759848]]


array([[[0.56620119, 0.08906015],
        [0.24706707, 0.18581138]],

       [[0.02074013, 0.10316636],
        [0.30929114, 0.36826331]],

       [[0.23826158, 0.23864747],
        [0.32624601, 0.52732461]],

       [[0.22075985, 0.93674687],
        [0.492622  , 0.31759848]]])